In [533]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import optuna
import lightgbm as lgb

df_train = pd.read_csv(('train.csv'))
df_test = pd.read_csv('test.csv')
census = pd.read_csv('census_starter.csv', index_col='cfips')
df_merged_df = pd.read_csv('sample_submission.csv')
df_new = pd.read_csv('revealed_test.csv')

In [534]:
df_train = pd.concat([df_train, df_new]).sort_values(by=['cfips', 'first_day_of_month']).reset_index()
drop_index = (df_test.first_day_of_month == '2022-11-01') | (df_test.first_day_of_month == '2022-12-01')
df_test = df_test.loc[~ drop_index]

In [535]:
df_train['is_train'] = 1
df_test['is_train'] = 0

In [536]:
state_dict = df_train[['cfips', 'state', 'county']]
state_dict = state_dict.set_index('cfips')
state_dict = state_dict.drop_duplicates()
state_dict = state_dict.to_dict()

df_test['state'] = df_test['cfips'].map(state_dict['state'])
df_test['county'] = df_test['cfips'].map(state_dict['county'])
zero_mask = (df_train['microbusiness_density'] == 0) & (df_train['active'] == 0) 
df_train['ratio'] = df_train.loc[~ zero_mask].groupby('cfips', group_keys=False).apply(lambda x: x.active / x.microbusiness_density)
df_train['ratio'] = df_train['ratio'].fillna(0)
df = pd.concat([df_train, df_test], axis=0)
df.state = df['state'].astype('category').cat.codes
df = df.drop(columns=['county'])

In [537]:
df.first_day_of_month = pd.to_datetime(df.first_day_of_month)
df['month'] = df.first_day_of_month.dt.month
df['year'] = df.first_day_of_month.dt.year - 2019
df['is_new_year'] = 0
df.loc[df.month == 1, 'is_new_year'] = 1
seasons = {1: 'Winter', 2: 'Winter', 3: 'Spring', 4: 'Spring', 5: 'Spring', 6: 'Summer', 7: 'Summer', 8: 'Summer', 9: 'Fall', 10: 'Fall', 11: 'Fall', 12: 'Winter'}
df['season'] = df['month'].apply(lambda x: seasons[x])
df['md_national_avg'] = df.groupby(['year','month'])['microbusiness_density'].transform('mean')
df['md_state_avg'] = df.groupby(['state','year','month'])['microbusiness_density'].transform('mean')
df['a_national_avg'] = df.groupby(['year','month'])['active'].transform('mean')
df['a_state_avg'] = df.groupby(['state','year','month'])['active'].transform('mean')


In [538]:
df = pd.concat([df, pd.get_dummies(df['season'], prefix='season')], axis=1)
df = df.drop(columns='season')

In [539]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

# Create a US holiday calendar
cal = USFederalHolidayCalendar()

holidays = cal.holidays(start='2019-01-01', end='2023-06-01')
df['holidays'] = pd.to_datetime(df['first_day_of_month'], format='%Y-%m-%d').dt.to_period('M').apply(lambda x: len(holidays[holidays.to_period('M') == x]))

In [540]:
df['idx'] = df.groupby('cfips')['cfips'].cumcount()
df = df.merge(census, on='cfips')

In [541]:
import matplotlib.pyplot as plt
%matplotlib inline

df = df.set_index('row_id')
mask = ((df.microbusiness_density == 0) & (df.active == 0))

df['md_state_avg_log_diff'] = df.loc[~ mask].groupby('cfips', group_keys=False)['md_state_avg'].apply(lambda x: np.log(x).diff())
df.loc[mask, [f'md_state_avg_log_diff']] = 0
df[f'md_state_avg_log_diff'] = df.loc[~ mask].groupby('cfips', group_keys=False)['md_state_avg_log_diff'].bfill()# is backfill needed?

df['a_state_avg_log_diff'] = df.loc[~ mask].groupby('cfips', group_keys=False)['a_state_avg'].apply(lambda x: np.log(x).diff())
df.loc[mask, [f'a_state_avg_log_diff']] = 0
df[f'a_state_avg_log_diff'] = df.loc[~ mask].groupby('cfips', group_keys=False)['a_state_avg_log_diff'].bfill()# is backfill needed?

LAGS = [1, 2, 3, 4, 5, 6, 7, 8]
for i in LAGS:
    df[f'md_log_diff_{i}'] = df.loc[~ mask].groupby('cfips', group_keys=False)['microbusiness_density'].apply(lambda x: np.log(x).diff())
    df.loc[mask, [f'md_log_diff_{i}' for i in LAGS]] = 0
    df[f'md_log_diff_{i}'] = df.loc[~ mask].groupby('cfips', group_keys=False)[f'md_log_diff_{i}'].bfill()# is backfill needed?
    df[f'a_log_diff_{i}'] = df.loc[~ mask].groupby('cfips', group_keys=False)['active'].apply(lambda x: np.log(x).diff())
    df.loc[mask, [f'a_log_diff_{i}' for i in LAGS]] = 0
    df[f'a_log_diff_{i}'] = df.loc[~ mask].groupby('cfips', group_keys=False)[f'a_log_diff_{i}'].bfill() # is backfill needed?
    df[f'md_lag_{i}'] = df.groupby('cfips', group_keys=False)['microbusiness_density'].shift(i)
    df[f'a_lag_{i}'] = df.groupby('cfips', group_keys=False)['microbusiness_density'].shift(i)
    
df['target'] = df.loc[~ mask].groupby('cfips', group_keys=False)['microbusiness_density'].apply(lambda x: np.log(x.shift(-5)) - np.log(x))
df.loc[mask, ['target']] = 0

In [542]:
year_mask = True
if year_mask:
    mask = (df['year'] == 1) | (df['year'] == 0)
    df = df.loc[~mask]
df

,index,cfips,state,first_day_of_month,microbusiness_density,active,is_train,ratio,month,year,...,a_lag_4,md_lag_5,a_lag_5,md_lag_6,a_lag_6,md_lag_7,a_lag_7,md_lag_8,a_lag_8,target
row_id,,,,,,,,,,,,,,,,,,,,,
1001_2021-01-01,17.0,1001,0,2021-01-01,2.947244,1243.0,1,421.749990,1,2,...,3.205756,3.174679,3.174679,3.083837,3.083837,3.019292,3.019292,3.004948,3.004948,0.089221
1001_2021-02-01,18.0,1001,0,2021-02-01,3.106106,1310.0,1,421.749988,2,2,...,3.193804,3.205756,3.205756,3.174679,3.174679,3.083837,3.083837,3.019292,3.019292,0.033036
1001_2021-03-01,19.0,1001,0,2021-03-01,3.144043,1326.0,1,421.749997,3,2,...,3.038416,3.193804,3.193804,3.205756,3.205756,3.174679,3.174679,3.083837,3.083837,0.023846
1001_2021-04-01,20.0,1001,0,2021-04-01,3.224659,1360.0,1,421.749995,4,2,...,3.002558,3.038416,3.038416,3.193804,3.193804,3.205756,3.205756,3.174679,3.174679,-0.011834
1001_2021-05-01,21.0,1001,0,2021-05-01,3.227030,1361.0,1,421.749991,5,2,...,2.947244,3.002558,3.002558,3.038416,3.038416,3.193804,3.193804,3.205756,3.205756,-0.007375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56045_2023-02-01,NaN,56045,50,2023-02-01,NaN,NaN,0,NaN,2,4,...,1.785395,1.785395,1.785395,1.785395,1.785395,1.803249,1.803249,1.803249,1.803249,NaN
56045_2023-03-01,NaN,56045,50,2023-03-01,NaN,NaN,0,NaN,3,4,...,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.803249,1.803249,NaN
56045_2023-04-01,NaN,56045,50,2023-04-01,NaN,NaN,0,NaN,4,4,...,1.803249,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,1.785395,NaN


In [543]:
df = df.to_csv('base_df.csv')

In [544]:
df = pd.read_csv('base_df.csv', index_col='row_id')

In [545]:
# reorganize train test
df_train = df[df.is_train == 1]
df_train = df_train.fillna(0)
df_test = df[df.is_train == 0]
target_train = df[df.is_train == 1]['target']
target_train = target_train.fillna(0)

In [546]:
from sklearn.neighbors import NearestNeighbors
from tqdm import tqdm

def get_NN_cfips_by_feats(df, N, p, metric, feature):
    df = df.reset_index()
    NN_feature = ['cfips', 'first_day_of_month', feature]
    df = df[NN_feature]
    df = df.pivot(index='cfips', columns='first_day_of_month', values=feature)
    df.columns.name = None
    nn = NearestNeighbors(
                n_neighbors=N, 
                p=p, 
                metric=metric, 
                # metric_params=metric_params
            )
    nn.fit(df)
    neighbors = nn.kneighbors(df, return_distance=False)
    df_nn = pd.DataFrame(neighbors, index=df.index)
    df_nn = df_nn.apply(lambda col: df.index[col])
    df_nn = df_nn.iloc[:, 1:]
    return df_nn

def get_NN_cfips_by_census(census, N, p, metric):
    nn_census= NearestNeighbors(
                n_neighbors=N, 
                p=p, 
                metric=metric, 
                # metric_params=metric_params
            )
    census_na = census[census.isna().any(axis=1)]
    na_mask = census_na.index
    census.loc[na_mask] = census.loc[na_mask].fillna(method='ffill', axis=1)
    df_mask = df.cfips.unique()
    census = census.loc[df_mask]
    nn_census.fit(census)
    neighbors = nn_census.kneighbors(census, return_distance=False)
    df_nn = pd.DataFrame(neighbors, index=census.index)
    df_nn = df_nn.iloc[:, 1:]
    df_nn= df_nn.loc[df_mask]
    df_nn = df_nn.rename(columns={1: 'NN_1', 2: 'NN_2', 3: 'NN_3'})
    df_nn[['NN_1', 'NN_2', 'NN_3']] = df_nn[['NN_1', 'NN_2', 'NN_3']].applymap(lambda x: census.index[x])
    return df_nn

def generate_feature(df_nn, df, feature, colname):
    cfips = df.cfips.unique()
    df_grouped = df.groupby('cfips')
    dfs = []
    for cfips_val in tqdm(cfips):
        nn_list = df_nn.loc[cfips_val].values.tolist()
        feats = pd.DataFrame(np.asarray([df_grouped.get_group(i)[feature].values for i in nn_list]).T, columns=[f'{colname}_NN_{i}' for i in range(len(nn_list))])
        # print(nn_list)
        # print(feats)
        dfs.append(feats)
    result = pd.concat(dfs, axis=0)
    result = result.reset_index()
    result = result.drop(columns='index')
    return result

def combine_nn_feats(df, df_nn):
    df = df.reset_index()
    df = pd.concat([df, df_nn], axis=1)
    df = df.drop(columns='first_day_of_month')
    df = df.set_index('row_id')
    return df

In [547]:
# df_nn_c = get_NN_cfips_by_census(census, N=4, p=2)
# df_nn_c
# generate_feature(df_nn_c, df_train, 'microbusiness_density', 'md')

In [548]:
KNN = True

if KNN:
    # NN by md:
    df_nn_c1 = get_NN_cfips_by_census(census, N=4, p=1, metric='manhattan')
    df_nn_1 = get_NN_cfips_by_feats(df_train, N=4, p=1, metric='manhattan', feature='microbusiness_density')
    md_log_diff_1_p1 = generate_feature(df_nn_1, df_train, 'md_log_diff_1', 'md_log_diff_1_1')
    md_log_diff_2_p1 = generate_feature(df_nn_1, df_train, 'md_log_diff_2', 'md_log_diff_2_1')
    md_log_diff_3_p1 = generate_feature(df_nn_1, df_train, 'md_log_diff_3', 'md_log_diff_3_1')
    md_log_diff_4_p1 = generate_feature(df_nn_1, df_train, 'md_log_diff_4', 'md_log_diff_4_1')
    md_log_diff_5_p1 = generate_feature(df_nn_1, df_train, 'md_log_diff_5', 'md_log_diff_5_1')
    md_state_avg_diff_c_p1 = generate_feature(df_nn_c1, df_train, 'md_state_avg_log_diff', 'md_state_avg_c_1')
    md_state_avg_diff_p1 = generate_feature(df_nn_1, df_train, 'md_state_avg_log_diff', 'md_state_avg_1')
    a_state_avg_diff_c_p1 = generate_feature(df_nn_c1, df_train, 'a_state_avg_log_diff', 'a_state_avg_c_1')
    a_state_avg_diff_p1 = generate_feature(df_nn_1, df_train, 'a_state_avg_log_diff', 'a_state_avg_1')
    md_lag_1_c_p1 = generate_feature(df_nn_c1, df_train, 'md_lag_1', 'lag_1_c_1')
    md_lag_2_c_p1 = generate_feature(df_nn_c1, df_train, 'md_lag_2', 'lag_2_c_1')
    md_lag_3_c_p1 = generate_feature(df_nn_c1, df_train, 'md_lag_3', 'lag_3_c_1')
    md_lag_4_c_p1 = generate_feature(df_nn_c1, df_train, 'md_lag_4', 'lag_4_c_1')
    md_lag_5_c_p1 = generate_feature(df_nn_c1, df_train, 'md_lag_5', 'lag_5_c_1')
    md_lag_1_p1 = generate_feature(df_nn_1, df_train, 'md_lag_1', 'lag_1_1')
    md_lag_2_p1 = generate_feature(df_nn_1, df_train, 'md_lag_2', 'lag_2_1')
    md_lag_3_p1 = generate_feature(df_nn_1, df_train, 'md_lag_3', 'lag_3_1')
    md_lag_4_p1 = generate_feature(df_nn_1, df_train, 'md_lag_4', 'lag_4_1')
    md_lag_5_p1 = generate_feature(df_nn_1, df_train, 'md_lag_5', 'lag_5_1')

    # NN by log_diff:
    df_nn_c2 = get_NN_cfips_by_census(census, N=4, p=2, metric='euclidean')
    df_nn_2 = get_NN_cfips_by_feats(df_train, N=4, p=2, metric = 'euclidean', feature='microbusiness_density')
    md_log_diff_1_p2 = generate_feature(df_nn_2, df_train, 'md_log_diff_1', 'md_log_diff_1_2')
    md_log_diff_2_p2 = generate_feature(df_nn_2, df_train, 'md_log_diff_2', 'md_log_diff_2_2')
    md_log_diff_3_p2 = generate_feature(df_nn_2, df_train, 'md_log_diff_3', 'md_log_diff_3_2')
    md_log_diff_4_p2 = generate_feature(df_nn_2, df_train, 'md_log_diff_4', 'md_log_diff_4_2')
    md_log_diff_5_p2 = generate_feature(df_nn_2, df_train, 'md_log_diff_5', 'md_log_diff_5_2')
    md_state_avg_diff_c_p2 = generate_feature(df_nn_c2, df_train, 'md_state_avg_log_diff', 'md_state_avg_c_2')
    md_state_avg_diff_p2 = generate_feature(df_nn_2, df_train, 'md_state_avg_log_diff', 'md_state_avg_2')
    a_state_avg_diff_c_p2 = generate_feature(df_nn_c2, df_train, 'a_state_avg_log_diff', 'a_state_avg_c_2')
    a_state_avg_diff_p2 = generate_feature(df_nn_2, df_train, 'a_state_avg_log_diff', 'a_state_avg_2')
    md_lag_1_c_p2 = generate_feature(df_nn_c2, df_train, 'md_lag_1', 'lag_1_c_2')
    md_lag_2_c_p2 = generate_feature(df_nn_c2, df_train, 'md_lag_2', 'lag_2_c_2')
    md_lag_3_c_p2 = generate_feature(df_nn_c2, df_train, 'md_lag_3', 'lag_3_c_2')
    md_lag_4_c_p2 = generate_feature(df_nn_c2, df_train, 'md_lag_4', 'lag_4_c_2')
    md_lag_5_c_p2 = generate_feature(df_nn_c2, df_train, 'md_lag_5', 'lag_5_c_2')
    md_lag_1_p2 = generate_feature(df_nn_2, df_train, 'md_lag_1', 'lag_1_2')
    md_lag_2_p2 = generate_feature(df_nn_2, df_train, 'md_lag_2', 'lag_2_2')
    md_lag_3_p2 = generate_feature(df_nn_2, df_train, 'md_lag_3', 'lag_3_2')
    md_lag_4_p2 = generate_feature(df_nn_2, df_train, 'md_lag_4', 'lag_4_2')
    md_lag_5_p2 = generate_feature(df_nn_2, df_train, 'md_lag_5', 'lag_5_2')
    NN_feats_list = [
        md_state_avg_diff_c_p1,
        md_state_avg_diff_c_p2,
        a_state_avg_diff_c_p1,
        a_state_avg_diff_c_p2,
        md_lag_1_c_p1,
        md_lag_1_c_p2,
        md_lag_2_c_p1,
        md_lag_2_c_p2,
        md_lag_3_c_p1,
        md_lag_3_c_p2,
        md_lag_4_c_p1,
        md_lag_4_c_p2,
        md_lag_5_c_p1,
        md_lag_5_c_p2,
        md_state_avg_diff_p1, 
        md_state_avg_diff_p2,
        a_state_avg_diff_p1,
        a_state_avg_diff_p2,
        md_lag_1_p1,
        md_lag_1_p2,
        md_lag_2_p1,
        md_lag_2_p2,
        md_lag_3_p1,
        md_lag_3_p2,
        md_lag_4_p1,
        md_lag_4_p2,
        md_lag_5_p1,
        md_lag_5_p2,
        md_log_diff_1_p1,
        md_log_diff_1_p2,
        md_log_diff_2_p1,
        md_log_diff_2_p2,
        md_log_diff_3_p1,
        md_log_diff_3_p2,
        md_log_diff_4_p1,
        md_log_diff_4_p2,
        md_log_diff_5_p1,
        md_log_diff_5_p2,
        ]
    NN_feats = pd.concat(NN_feats_list, axis=1)
    df_train = combine_nn_feats(df_train, NN_feats)
else:
    df_train = df_train.drop(columns='first_day_of_month')

100%|██████████| 3135/3135 [00:00<00:00, 3938.30it/s]


In [549]:
df_train

,index,cfips,state,microbusiness_density,active,is_train,ratio,month,year,is_new_year,...,md_log_diff_4_1_NN_2,md_log_diff_4_2_NN_0,md_log_diff_4_2_NN_1,md_log_diff_4_2_NN_2,md_log_diff_5_1_NN_0,md_log_diff_5_1_NN_1,md_log_diff_5_1_NN_2,md_log_diff_5_2_NN_0,md_log_diff_5_2_NN_1,md_log_diff_5_2_NN_2
row_id,,,,,,,,,,,,,,,,,,,,,
1001_2021-01-01,17.0,1001,0,2.947244,1243.0,1,421.749990,1,2,1,...,0.014377,0.005454,0.006040,-0.012909,0.003468,0.005454,0.014377,0.005454,0.006040,-0.012909
1001_2021-02-01,18.0,1001,0,3.106106,1310.0,1,421.749988,2,2,0,...,0.004673,0.017133,0.053859,-0.022776,-0.012601,0.017133,0.004673,0.017133,0.053859,-0.022776
1001_2021-03-01,19.0,1001,0,3.144043,1326.0,1,421.749997,3,2,0,...,0.013889,0.014644,0.007463,0.005158,0.010811,0.014644,0.013889,0.014644,0.007463,0.005158
1001_2021-04-01,20.0,1001,0,3.224659,1360.0,1,421.749995,4,2,0,...,-0.002302,0.004396,0.000000,0.010237,0.000000,0.004396,-0.002302,0.004396,0.000000,0.010237
1001_2021-05-01,21.0,1001,0,3.227030,1361.0,1,421.749991,5,2,0,...,-0.006936,0.000000,0.014760,-0.003711,-0.001794,0.000000,-0.006936,0.000000,0.014760,-0.003711
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56045_2022-08-01,122262.0,56045,50,1.785395,100.0,1,56.009999,8,3,0,...,0.000854,0.002457,-0.015198,-0.016807,-0.015198,-0.018405,0.000854,0.002457,-0.015198,-0.016807
56045_2022-09-01,122263.0,56045,50,1.785395,100.0,1,56.009999,9,3,0,...,0.009342,0.008552,-0.006144,-0.008511,-0.006144,0.000000,0.009342,0.008552,-0.006144,-0.008511
56045_2022-10-01,122264.0,56045,50,1.785395,100.0,1,56.009999,10,3,0,...,0.010093,-0.011009,0.006144,-0.008584,0.006144,0.000000,0.010093,-0.011009,0.006144,-0.008584


In [550]:
# df_test = df_test.drop(columns='first_day_of_month')
# df = pd.concat([df_train, df_test], axis=0)

In [551]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import TimeSeriesSplit

def smape_loss(y_true, y_pred):
    """
    Custom SMAPE loss function for LightGBM
    """
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

def smape_cv(model, df_train, target_train):
    tscv = TimeSeriesSplit(n_splits=5)
    smape_list = []
    for _, (train_index, test_index) in enumerate(tscv.split(df_train), start=1):
        # Create training and validation datasets
        X_train = df_train.iloc[train_index]
        y_train = target_train.iloc[train_index]
        X_valid = df_train.iloc[test_index]
        y_valid = target_train.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train)

        # Make predictions
        y_pred = np.exp(model.predict(X_valid)) * X_valid['microbusiness_density']
        y_valid = np.exp(y_valid) * X_valid['microbusiness_density']

        # Calculate SMAPE score
        smape = smape_loss(y_valid, y_pred)
        smape_list.append(smape)
    
    print(f'Mean SMAPE: {np.mean(smape_list):.4f}')
    return np.mean(smape_list)

def lgbm_objective(trial, df_train, target_train):
    """
    Objective function for LightGBM optimization without rolling window cross-validation.
    """
    params = {
        'n_iter'           : 200,
        'verbosity'        : -1,
        'objective'        : 'l1',
        'random_state'     : 42,
        'extra_trees'      : True,
        'colsample_bytree' : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode' : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth'        : trial.suggest_int('max_depth', 3, 10),
        'learning_rate'    : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'lambda_l1'        : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'        : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves'       : trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf' : trial.suggest_int('min_data_in_leaf', 5, 250),}

    model = lgb.LGBMRegressor(**params)
    score = smape_cv(model, df_train, target_train)
    return score


In [553]:
import optuna
import pandas as pd

# Load your data into df_train and target_train
# df_train = ...
# target_train = ...

# Create the study
study = optuna.create_study(direction='minimize')

# Optimize the study
study.optimize(lambda trial: lgbm_objective(trial, df_train, target_train), n_trials=20)

# Print the best trial
print(f"Best trial: {study.best_trial.value} with parameters {study.best_trial.params}")


[I 2023-03-14 19:31:00,525] A new study created in memory with name: no-name-94532758-556b-4cdb-b52a-ca376201e263
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
[LightGBM] [Warning] min_data_in_leaf is set=228, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=228
[LightGBM] [Warning] lambda_l2 is set=9.663972207751238, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.663972207751238
[LightGBM] [Warning] lambda_l1 is set=8.798090798420677, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.798090798420677


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
[LightGBM] [Warning] min_data_in_leaf is set=228, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=228
[LightGBM] [Warning] lambda_l2 is set=9.663972207751238, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.663972207751238
[LightGBM] [Warning] lambda_l1 is set=8.798090798420677, reg_alpha=0.0 will be ignored. Current value: lambda_l1=8.798090798420677


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
[LightGBM] [Warning] min_data_in_leaf is set=166, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=166
[LightGBM] [Warning] lambda_l2 is set=1.2552769958120122, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2552769958120122
[LightGBM] [Warning] lambda_l1 is set=3.817536482492845, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.817536482492845


/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_iter` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] num_iterations is set=200, n_iter=200 will be ignored. Current value: num_iterations=200
[LightGBM] [Warning] min_data_in_leaf is set=166, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=166
[LightGBM] [Warning] lambda_l2 is set=1.2552769958120122, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.2552769958120122
[LightGBM] [Warning] lambda_l1 is set=3.817536482492845, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.817536482492845


[I 2023-03-14 19:36:02,758] Trial 19 finished with value: 0.5430054714553861 and parameters: {'colsample_bytree': 0.876672240265486, 'colsample_bynode': 0.8643191960002234, 'max_depth': 9, 'learning_rate': 0.07277889651307486, 'lambda_l1': 3.817536482492845, 'lambda_l2': 1.2552769958120122, 'num_leaves': 786, 'min_data_in_leaf': 166}. Best is trial 15 with value: 0.49877623401352195.


Mean SMAPE: 0.5430
Best trial: 0.49877623401352195 with parameters {'colsample_bytree': 0.922550834662478, 'colsample_bynode': 0.8865208494474415, 'max_depth': 9, 'learning_rate': 0.09982813819054616, 'lambda_l1': 4.135437333012513, 'lambda_l2': 1.4608779450861495, 'num_leaves': 539, 'min_data_in_leaf': 189}


In [567]:
import xgboost as xgb

def xgb_objective(trial, df_train, target_train):
    """
    Objective function for XGBoost optimization without rolling window cross-validation.
    """
    params = {
        'n_estimators': 200,
        'verbosity': 0,
        'objective': 'reg:squarederror',
        'random_state': 42,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'alpha': trial.suggest_float('alpha', 1e-2, 10.0),
        'lambda': trial.suggest_float('lambda', 1e-2, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 5, 250),
    }

    model = xgb.XGBRegressor(**params)
    score = smape_cv(model, df_train, target_train)
    return score


In [569]:
# Create the study
study = optuna.create_study(direction='minimize')

# Optimize the study
study.optimize(lambda trial: xgb_objective(trial, df_train, target_train), n_trials=30)

# Print the best trial
print(f"Best trial: {study.best_trial.value} with parameters {study.best_trial.params}")


[I 2023-03-14 19:38:17,243] A new study created in memory with name: no-name-6bced3e3-1f87-4326-8537-55141f03b4db
[I 2023-03-14 19:38:37,577] Trial 0 finished with value: 2.704374945494864 and parameters: {'colsample_bytree': 0.4682941391932415, 'colsample_bynode': 0.6982854631495193, 'max_depth': 8, 'learning_rate': 0.015216171491942062, 'alpha': 9.758805941563816, 'lambda': 0.4437882972622384, 'min_child_weight': 22}. Best is trial 0 with value: 2.704374945494864.


Mean SMAPE: 2.7044


[I 2023-03-14 19:38:52,449] Trial 1 finished with value: 1.0140977498720574 and parameters: {'colsample_bytree': 0.412737287458086, 'colsample_bynode': 0.24590672717561096, 'max_depth': 6, 'learning_rate': 0.045272347755726104, 'alpha': 9.088250070490984, 'lambda': 1.9219035329453777, 'min_child_weight': 7}. Best is trial 1 with value: 1.0140977498720574.


Mean SMAPE: 1.0141


[I 2023-03-14 19:39:11,892] Trial 2 finished with value: 0.634442486787863 and parameters: {'colsample_bytree': 0.34220562707296787, 'colsample_bynode': 0.5125269664182217, 'max_depth': 6, 'learning_rate': 0.05499796909602932, 'alpha': 3.3487545746642224, 'lambda': 2.97233068127703, 'min_child_weight': 164}. Best is trial 2 with value: 0.634442486787863.


...

Mean SMAPE: 0.3699


[I 2023-03-14 19:52:02,547] Trial 22 finished with value: 0.4246261860663479 and parameters: {'colsample_bytree': 0.6755389950457995, 'colsample_bynode': 0.7937964175799626, 'max_depth': 9, 'learning_rate': 0.02992040013461166, 'alpha': 4.830671050216401, 'lambda': 6.37718516440858, 'min_child_weight': 90}. Best is trial 18 with value: 0.315527144979913.


Mean SMAPE: 0.4246


[W 2023-03-14 19:52:06,134] Trial 23 failed with parameters: {'colsample_bytree': 0.5208313996967846, 'colsample_bynode': 0.9120042568617732, 'max_depth': 7, 'learning_rate': 0.04413552365828709, 'alpha': 7.725905284934811, 'lambda': 5.1487578637215226, 'min_child_weight': 65} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/bb/g8c6d14n37lb8f846cg2t1_m0000gn/T/ipykernel_28597/2937589560.py", line 5, in <lambda>
    study.optimize(lambda trial: xgb_objective(trial, df_train, target_train), n_trials=30)
  File "/var/folders/bb/g8c6d14n37lb8f846cg2t1_m0000gn/T/ipykernel_28597/3957172496.py", line 22, in xgb_objective
    score = smape_cv(model, df_train, target_train)
  File "/var/folders/bb/g8c6d14n37lb8f846cg2t1_m0000gn/T/ipykernel_28597/3551183721.py", line 21, in smape

KeyboardInterrupt: 

In [555]:
from sklearn.linear_model import LinearRegression, ElasticNet, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb


LGBM = lgb.LGBMRegressor(**params)

def smape_cv2(model):
    tscv = TimeSeriesSplit(n_splits=5)
    smape_list = []
    model_name = model.__class__.__name__
    for _, (train_index, test_index) in tqdm(enumerate(tscv.split(df_train), start=1), 
                                             desc=f'{model_name} Cross Validations', total=7):
        X_train, X_test = df_train.iloc[train_index], df_train.iloc[test_index]
        y_train, y_test = target_train.iloc[train_index], target_train.iloc[test_index]
        clf = model.fit(X_train, y_train)
        pred = clf.predict(X_test)
        pred =  np.exp(pred) * X_test['microbusiness_density']
        y_test = np.exp(y_test) * X_test['microbusiness_density']
        smape = smape_loss(y_test, pred) 
        smape_list.append(smape)
    return model_name, smape_list

def print_smape_score(model):
    # print cv and save the average
    model_name, score = smape_cv2(model)
    for i, r in enumerate(score, start=1):
        print(f'{i} FOLDS: {model_name} smape: {r:.4f}')
    print(f'\n{model_name} mean smape: {np.mean(score):.4f}')
    print('='*30)
    return model_name, np.mean(score)

models = []
scores = []
for model in [LGBM]:
    model_name, mean_score = print_smape_score(model)
    models.append(model_name)
    scores.append(mean_score)

LGBMRegressor Cross Validations:   0%|          | 0/7 [00:00<?, ?it/s]

[LightGBM] [Warning] min_data_in_leaf is set=189, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=189
[LightGBM] [Warning] lambda_l1 is set=4.135437333012513, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.135437333012513
[LightGBM] [Warning] lambda_l2 is set=1.4608779450861495, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4608779450861495


LGBMRegressor Cross Validations:  14%|█▍        | 1/7 [00:00<00:03,  1.62it/s]

[LightGBM] [Warning] min_data_in_leaf is set=189, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=189
[LightGBM] [Warning] lambda_l1 is set=4.135437333012513, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.135437333012513
[LightGBM] [Warning] lambda_l2 is set=1.4608779450861495, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4608779450861495


LGBMRegressor Cross Validations:  29%|██▊       | 2/7 [00:01<00:03,  1.33it/s]

...

[LightGBM] [Warning] min_data_in_leaf is set=189, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=189
[LightGBM] [Warning] lambda_l1 is set=4.135437333012513, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.135437333012513
[LightGBM] [Warning] lambda_l2 is set=1.4608779450861495, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4608779450861495


LGBMRegressor Cross Validations:  57%|█████▋    | 4/7 [00:03<00:03,  1.01s/it]

[LightGBM] [Warning] min_data_in_leaf is set=189, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=189
[LightGBM] [Warning] lambda_l1 is set=4.135437333012513, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.135437333012513
[LightGBM] [Warning] lambda_l2 is set=1.4608779450861495, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4608779450861495


LGBMRegressor Cross Validations:  71%|███████▏  | 5/7 [00:05<00:02,  1.08s/it]

1 FOLDS: LGBMRegressor smape: 0.7201
2 FOLDS: LGBMRegressor smape: 0.3819
3 FOLDS: LGBMRegressor smape: 0.5324
4 FOLDS: LGBMRegressor smape: 0.3208
5 FOLDS: LGBMRegressor smape: 0.3340

LGBMRegressor mean smape: 0.4578


In [213]:
model = xgb.XGBRegressor(**params)

In [556]:
x_test = df_train[df_train.idx == 40]
model.fit(df_train, target_train)

y_pred = np.exp(model.predict(x_test)) * x_test.microbusiness_density
y_true = np.exp(x_test.md_log_diff_3) * x_test.microbusiness_density
y_base = x_test.microbusiness_density

[LightGBM] [Warning] min_data_in_leaf is set=189, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=189
[LightGBM] [Warning] lambda_l1 is set=4.135437333012513, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4.135437333012513
[LightGBM] [Warning] lambda_l2 is set=1.4608779450861495, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.4608779450861495


In [557]:
y_pred = y_pred.reset_index('row_id')
y_base = y_base.reset_index()
y_base = y_base.rename(columns={'microbusiness_density': 'y_base'})
y_true = y_true.reset_index()
y_true = y_true.rename(columns={0: 'y_true'})

In [558]:
y_pred

,row_id,microbusiness_density
0,1001_2022-12-01,3.471754
1,1003_2022-12-01,8.249109
2,1005_2022-12-01,1.252116
3,1007_2022-12-01,1.287124
4,1009_2022-12-01,1.851950
...,...,...
3130,56037_2022-12-01,2.877461
3131,56039_2022-12-01,26.267223
3132,56041_2022-12-01,4.050750
3133,56043_2022-12-01,3.060968


In [559]:
clip_err = True

if clip_err:
    df_record = y_pred.merge(y_base, on = 'row_id', how='inner')
    df_record = df_record.merge(y_true, on = 'row_id', how='inner')
    def smape_(y_true, y_pred):
        return 100  * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
    base_err = df_record.apply(lambda x: smape_(x.y_true, x.y_base), axis=1)
    pred_err = df_record.apply(lambda x: smape_(x.y_true, x.microbusiness_density), axis=1)
    blacklist = base_err[(base_err + 1e-3) < pred_err].index
    y_pred.iloc[blacklist] = y_base.iloc[blacklist]
    avg_base_err = base_err.mean()
    avg_pred_err = pred_err.mean()
    print(avg_base_err, avg_pred_err)
    df_sub = pd.read_csv('sample_submission.csv')
    df_sub = df_sub.drop(columns='microbusiness_density')
    df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
    y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
    merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
    merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
    # merged_df = merged_df.drop(columns='cfips')
else:
    df_sub = pd.read_csv('sample_submission.csv')
    df_sub = df_sub.drop(columns='microbusiness_density')
    df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
    y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
    merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
    merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
    # merged_df = merged_df.drop(columns='cfips')

1.8284295961187311 1.8374568832227278


In [560]:
merged_df

,row_id,cfips,microbusiness_density
0,1001_2022-11-01,1001,3.471754
1,1003_2022-11-01,1003,8.249109
2,1005_2022-11-01,1005,1.252272
3,1007_2022-11-01,1007,1.287240
4,1009_2022-11-01,1009,1.852060
...,...,...,...
25075,56037_2023-06-01,56037,2.877461
25076,56039_2023-06-01,56039,26.267223
25077,56041_2023-06-01,56041,4.050750
25078,56043_2023-06-01,56043,3.060380


In [561]:
COLS = ['GEO_ID','NAME','S0101_C01_026E']
df2020 = pd.read_csv('ACSST5Y2020.S0101-Data.csv', usecols=COLS)
df2020 = df2020.iloc[1:]
df2020['S0101_C01_026E'] = df2020['S0101_C01_026E'].astype('int')
df2020


/var/folders/bb/g8c6d14n37lb8f846cg2t1_m0000gn/T/ipykernel_28597/2667666172.py:2: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df2020 = pd.read_csv('ACSST5Y2020.S0101-Data.csv', usecols=COLS)


,GEO_ID,NAME,S0101_C01_026E
1,0500000US01001,"Autauga County, Alabama",42496
2,0500000US01003,"Baldwin County, Alabama",171296
3,0500000US01005,"Barbour County, Alabama",19804
4,0500000US01007,"Bibb County, Alabama",17790
5,0500000US01009,"Blount County, Alabama",44383
...,...,...,...
3217,0500000US72145,"Vega Baja Municipio, Puerto Rico",41383
3218,0500000US72147,"Vieques Municipio, Puerto Rico",6884
3219,0500000US72149,"Villalba Municipio, Puerto Rico",17344
3220,0500000US72151,"Yabucoa Municipio, Puerto Rico",26803


In [562]:
df2021 = pd.read_csv('ACSST5Y2021.S0101-Data.csv', usecols=COLS)
df2021 = df2021.iloc[1:]
df2021['S0101_C01_026E'] = df2021['S0101_C01_026E'].astype('int')
df2021

/var/folders/bb/g8c6d14n37lb8f846cg2t1_m0000gn/T/ipykernel_28597/1346287853.py:1: DtypeWarning: Columns (102) have mixed types. Specify dtype option on import or set low_memory=False.
  df2021 = pd.read_csv('ACSST5Y2021.S0101-Data.csv', usecols=COLS)


,GEO_ID,NAME,S0101_C01_026E
1,0500000US01001,"Autauga County, Alabama",44438
2,0500000US01003,"Baldwin County, Alabama",178105
3,0500000US01005,"Barbour County, Alabama",19995
4,0500000US01007,"Bibb County, Alabama",17800
5,0500000US01009,"Blount County, Alabama",45201
...,...,...,...
3217,0500000US72145,"Vega Baja Municipio, Puerto Rico",44604
3218,0500000US72147,"Vieques Municipio, Puerto Rico",6760
3219,0500000US72149,"Villalba Municipio, Puerto Rico",17811
3220,0500000US72151,"Yabucoa Municipio, Puerto Rico",25533


In [563]:
df2020['cfips'] = df2020.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
adult2020 = df2020.set_index('cfips').S0101_C01_026E.to_dict()

df2021['cfips'] = df2021.GEO_ID.apply(lambda x: int(x.split('US')[-1]) )
adult2021 = df2021.set_index('cfips').S0101_C01_026E.to_dict()

In [564]:
df2020

,GEO_ID,NAME,S0101_C01_026E,cfips
1,0500000US01001,"Autauga County, Alabama",42496,1001
2,0500000US01003,"Baldwin County, Alabama",171296,1003
3,0500000US01005,"Barbour County, Alabama",19804,1005
4,0500000US01007,"Bibb County, Alabama",17790,1007
5,0500000US01009,"Blount County, Alabama",44383,1009
...,...,...,...,...
3217,0500000US72145,"Vega Baja Municipio, Puerto Rico",41383,72145
3218,0500000US72147,"Vieques Municipio, Puerto Rico",6884,72147
3219,0500000US72149,"Villalba Municipio, Puerto Rico",17344,72149
3220,0500000US72151,"Yabucoa Municipio, Puerto Rico",26803,72151


In [565]:
merged_df.cfips = merged_df.cfips.astype(int)
merged_df['adult2020'] = merged_df.cfips.map(adult2020)
merged_df['adult2021'] = merged_df.cfips.map(adult2021)
merged_df

,row_id,cfips,microbusiness_density,adult2020,adult2021
0,1001_2022-11-01,1001,3.471754,42496,44438
1,1003_2022-11-01,1003,8.249109,171296,178105
2,1005_2022-11-01,1005,1.252272,19804,19995
3,1007_2022-11-01,1007,1.287240,17790,17800
4,1009_2022-11-01,1009,1.852060,44383,45201
...,...,...,...,...,...
25075,56037_2023-06-01,56037,2.877461,32049,31514
25076,56039_2023-06-01,56039,26.267223,19164,19169
25077,56041_2023-06-01,56041,4.050750,14516,14641
25078,56043_2023-06-01,56043,3.060380,6045,6000


In [566]:
merged_df.microbusiness_density = merged_df.microbusiness_density * merged_df.adult2020 / merged_df.adult2021
merged_df = merged_df.drop(['adult2020','adult2021','cfips'], axis=1)
merged_df.to_csv('lgbm_MAY.csv',index=False)
merged_df.head()

,row_id,microbusiness_density
0,1001_2022-11-01,3.320034
1,1003_2022-11-01,7.933743
2,1005_2022-11-01,1.240310
3,1007_2022-11-01,1.286517
4,1009_2022-11-01,1.818544
